# DEAP fuzzy chromosome experimentation
Experimenting with creating a chromosome to represent a fuzzy rule.

In [1]:
from deap import creator, base, algorithms, gp, tools
import random
import matplotlib.pyplot as plt
import seaborn as sns

import skfuzzy as fuzz
from skfuzzy import control as ctrl
from skfuzzy.control import Rule 
from skfuzzy.control.term import Term
import random
import operator
import numpy as np


# Define the primitive set for a fuzzy rule

In [2]:
def noop(x): return x

def makePrimitiveSet(antecendents, consequents):
    pset = gp.PrimitiveSetTyped("Rule", [], Rule)
    for ant in antecendents:
        for name, term in ant.terms.items(): 
            pset.addTerminal(term, Term, f"{ant.label}_{name}")
    pset.addEphemeralConstant("consequents", lambda: [random.choice(consequents)], list)
    pset.addPrimitive(Rule, [Term, list], Rule)
    pset.addPrimitive(operator.and_, [Term, Term], Term)
    pset.addPrimitive(operator.or_, [Term, Term], Term)
    pset.addPrimitive(operator.invert, [Term], Term)
    pset.addPrimitive(noop, [list], list)
    return pset


# Create the Antecedents and Consequents

In [3]:
petal_length = ctrl.Antecedent(np.linspace(1.0, 7.0, 11), 'petal_length')
petal_width = ctrl.Antecedent(np.linspace(0.0, 2.5, 11), 'petal_width')
petal_length.automf(names="short medium long".split())
petal_width.automf(names="narrow medium wide".split())

setosa = ctrl.Consequent(np.linspace(0, 1, 10), 'setosa', "som")
setosa['likely'] = fuzz.trimf(setosa.universe, (0., 1., 1.))

versicolor = ctrl.Consequent(np.linspace(0, 1, 10), 'versicolor', "som")
versicolor['likely'] = fuzz.trimf(versicolor.universe, (0., 1., 1.))

verginica = ctrl.Consequent(np.linspace(0, 1, 10), 'verginica', "som")
verginica['likely'] = fuzz.trimf(verginica.universe, (0., 1., 1.))

pset = makePrimitiveSet([petal_length, petal_width], [setosa, versicolor, verginica])


# Define constants 

In [4]:
POP_SIZE = 100
P_CROSSOVER = 0.9
P_MUTATION = 0.5
MAX_GENERATIONS = 30
HOF_SIZE = 10

MIN_TREE_HEIGHT = 3
MAX_TREE_HEIGHT = 6
MUT_MIN_TREE_HEIGHT = 0
MUT_MAX_TREE_HEIGHT = 2
LIMIT_TREE_HEIGHT = 17

# Define compile func, Individual class, select and mate operators etc

In [5]:
toolbox = base.Toolbox()

toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genGrow, min_=MUT_MIN_TREE_HEIGHT, max_=MUT_MAX_TREE_HEIGHT)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=LIMIT_TREE_HEIGHT))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=LIMIT_TREE_HEIGHT))

creator.create("FitnessMin", base.Fitness, weights=(-1.,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=MIN_TREE_HEIGHT, max_=MAX_TREE_HEIGHT)
toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)